## EV Charging & Registration Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
from scipy.stats import linregress
import statsmodels.api as sm
import matplotlib.cm as cm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from datetime import datetime
import matplotlib.ticker as ticker

ModuleNotFoundError: No module named 'matplotlib'

# International new EV registrations (2024)

In [ ]:
#Data from Statista

worldwide = pd.read_excel("datasets/statistic_id1220664_absatz-von-elektroautos-weltweit-nach-laendern-2024.xlsx")

country_translation = {
    'Deutschland': 'Germany', 'Vereinigtes Königreich': 'United Kingdom',
    'Frankreich': 'France', 'Kanada': 'Canada', 'Belgien': 'Belgium',
    'Niederlande': 'Netherlands', 'Schweden': 'Sweden', 'Südkorea': 'South Korea',
    'Norwegen': 'Norway', 'Spanien': 'Spain', 'Italien': 'Italy',
    'Dänemark': 'Denmark', 'Schweiz': 'Switzerland', 'Österreich': 'Austria',
    'Finnland': 'Finland', 'restliche Welt': 'Rest of World'
}
worldwide["Länder"] = worldwide["Länder"].map(country_translation).fillna(worldwide["Länder"])

worldwide.head()

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(worldwide["Länder"], worldwide["2024"])
plt.xticks(rotation=45, ha='right')
plt.xlabel("Country")
plt.ylabel("Registrations")
plt.title("Total new electric car registrations per country 2024")
ax = plt.gca()
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.tight_layout()
plt.show()

In [ ]:
worldwide['registrations_per_1000'] = (worldwide['2024'] / (worldwide['Einwohner'] * 1000000)) * 1000

worldwide_sorted = worldwide.sort_values('registrations_per_1000', ascending=False)

plt.figure(figsize=(12, 6))
bars = plt.bar(worldwide_sorted["Länder"], worldwide_sorted["registrations_per_1000"])
plt.xticks(rotation=45, ha='right')
plt.xlabel("Country")
plt.ylabel("Registrations per 1,000 inhabitants")
plt.title("Electric car registrations per capita by country 2024")

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2., height + 0.1, f'{height:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Charging station infrastructure & new registrations in Germany

In [ ]:
current_data = {
    'Fahrzeugtyp': ['Reine Elektroautos (BEV)', 'Plug-in Hybride (PHEV)', 'Gesamt EV + PHEV'],
    'Anzahl_Mio': [1.65, 0.97, 2.62],
    'Anteil_Prozent': [63.0, 37.0, 100.0]
}

charging_stations_total = 170000  # 170K Ladestationen

print("=== Aktuelle EV-Situation in Deutschland (2025) ===")
print(f"Reine Elektroautos (BEV): {current_data['Anzahl_Mio'][0]:.2f} Mio")
print(f"Plug-in Hybride (PHEV): {current_data['Anzahl_Mio'][1]:.2f} Mio") 
print(f"Gesamt EV + PHEV: {current_data['Anzahl_Mio'][2]:.2f} Mio")
print(f"Öffentliche Ladestationen: {charging_stations_total:,}")
print()
print("=== Verhältnisse ===")
print(f"EV Anteil: {current_data['Anteil_Prozent'][0]:.1f}%")
print(f"PHEV Anteil: {current_data['Anteil_Prozent'][1]:.1f}%")
print(f"Fahrzeuge pro Ladestation: {(current_data['Anzahl_Mio'][2] * 1000000) / charging_stations_total:.1f}")
print(f"Ladestationen pro 1000 Fahrzeuge: {(charging_stations_total / (current_data['Anzahl_Mio'][2] * 1000000)) * 1000:.1f}")
print()
print("=== EU Empfehlung ===")
print("1 Charging Station for 10 EVs")
print()
print("=== Goal in Germany ===")
print("1 Million Charging Stations und 15 Millionen Elektroautos till 2030 - realistic?")
print("Bis 2026 soll mindestens alle 60 Kilometer eine öffentliche Ladestation verfügbar sein. - Sind wir da schon?")


In [ ]:
current_data = {
    'Fahrzeugtyp': ['EV', 'PHEV', 'Total'],
    'Anzahl_Mio': [1.65, 0.97, 2.62],
    'Anteil_Prozent': [63.0, 37.0, 100.0]
}
charging_stations_total = 170000
vehicles_per_station = (current_data['Anzahl_Mio'][2] * 1000000) / charging_stations_total
eu_recommendation_ratio = 10

plt.figure()
labels = [current_data['Fahrzeugtyp'][0], current_data['Fahrzeugtyp'][1]]
sizes = [current_data['Anteil_Prozent'][0], current_data['Anteil_Prozent'][1]]
colors = ['blue', 'red']
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('EV Types Share (Germany 2025)')
plt.axis('equal')

plt.figure()
categories = ['EV', 'PHEV', 'Charging Points']
values_mio = [current_data['Anzahl_Mio'][0], current_data['Anzahl_Mio'][1], charging_stations_total / 1000000]
bar_colors = ['blue', 'red', 'orange']
bars = plt.bar(categories, values_mio, color=bar_colors)

for bar, cat in zip(bars, categories):
    yval = bar.get_height()
    if "Points" in cat:
         label_text = str(charging_stations_total)
         unit = ''
    else:
         label_text = str(round(yval, 2))
         unit = ' Mio.'
    plt.text(bar.get_x() + bar.get_width()/2, yval, label_text + unit, ha='center', va='bottom')

plt.ylabel('Amount in Millions')
plt.title('EVs vs Charging Points (Germany 2025)')

plt.figure()
ratio_categories = ['Current (GER)', 'EU Goal']
ratio_values = [vehicles_per_station, eu_recommendation_ratio]
ratio_colors = ['orange', 'green']
bars_ratio = plt.bar(ratio_categories, ratio_values, color=ratio_colors, width=0.5)

for bar in bars_ratio:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, str(round(yval, 1)), ha='center', va='bottom')

plt.ylabel('Vehicles per Charging Point')
plt.title('Vehicle/Point Ratio (GER vs EU Goal)')

plt.show()

In [ ]:
#Data from Statista

charging_ger = pd.read_excel("statistic_id1300745_number-of-new-public-charging-stations-germany-2014-2025.xlsx")
charging_ger.head()

In [ ]:
#Data from Statista

newregistration_electric_ger = pd.read_excel("statistic_id1166572_new-electric-car-registrations-in-germany-2003-2024.xlsx")
newregistration_electric_ger.head()

In [ ]:
#Data from Statista

newregistration_hybrid_ger = pd.read_excel("statistic_id1241597_pkw-neuzulassungen-mit-plug-in-hybrid-antrieb-in-deutschland-bis-september-2025.xlsx")
newregistration_hybrid_ger.head()

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.bar(charging_ger["Year"], charging_ger["Amount"])
plt.xlabel("Year")
plt.title("New public charging stations in Germany")

plt.subplot(1, 3, 2)
plt.bar(newregistration_electric_ger["year"], newregistration_electric_ger["amount"])
plt.xlabel("Year")
plt.title("New electric car registrations in Germany")

plt.subplot(1, 3, 3)
plt.bar(newregistration_hybrid_ger["year"], newregistration_hybrid_ger["amount"])
plt.xlabel("Year")
plt.title("New plug-in-hybrid car registrations in Germany")

plt.tight_layout()
plt.show()

In [ ]:
newregistration_monthly = pd.read_csv("ev_registrations_cleaned.csv")

newregistration_monthly.head()

In [ ]:
# Without political events

plt.figure(figsize=(15, 8))

newregistration_monthly['Date'] = pd.to_datetime(newregistration_monthly['Year'].astype(str) + '-' + newregistration_monthly['Month'].astype(str) + '-01')

electric_data = newregistration_monthly[newregistration_monthly['Type'] == 'Reine Elektroautos']
hybrid_data = newregistration_monthly[newregistration_monthly['Type'] == 'Plug-In Hybridautos']

plt.plot(electric_data['Date'], electric_data['Count'], 
         color='blue', marker='o', linewidth=2, markersize=4, label='Electric')
plt.plot(hybrid_data['Date'], hybrid_data['Count'], 
         color='red', marker='s', linewidth=2, markersize=4, label='Plug-in Hybrid')

plt.xlabel('Date')
plt.ylabel('Number of Registrations')
plt.title('Monthly New Car Registrations: Electric vs Plug-in Hybrid (Germany)')
plt.legend()

ax = plt.gca()

years = range(newregistration_monthly['Year'].min(), newregistration_monthly['Year'].max() + 1)
major_ticks = [pd.to_datetime(f'{year}-01-01') for year in years]
ax.set_xticks(major_ticks)
ax.set_xticklabels([str(year) for year in years])

from matplotlib.dates import MonthLocator, DateFormatter
ax.xaxis.set_minor_locator(MonthLocator())

ax.grid(True, which='major', alpha=0.6, linewidth=1)  
ax.grid(True, which='minor', alpha=0.3, linewidth=0.5)  

ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))

plt.xticks(rotation=0)  
plt.show()

In [ ]:
# With political events

plt.figure(figsize=(15, 8))

newregistration_monthly['Date'] = pd.to_datetime(newregistration_monthly['Year'].astype(str) + '-' + newregistration_monthly['Month'].astype(str) + '-01')

electric_data = newregistration_monthly[newregistration_monthly['Type'] == 'Reine Elektroautos']
hybrid_data = newregistration_monthly[newregistration_monthly['Type'] == 'Plug-In Hybridautos']

plt.plot(electric_data['Date'], electric_data['Count'], 
         color='blue', marker='o', linewidth=2, markersize=4, label='Electric')
plt.plot(hybrid_data['Date'], hybrid_data['Count'], 
         color='red', marker='s', linewidth=2, markersize=4, label='Plug-in Hybrid')

plt.xlabel('Date')
plt.ylabel('Number of Registrations')
plt.title('Monthly New Car Registrations: Electric vs Plug-in Hybrid (Germany)')
plt.legend()

ax = plt.gca()

years = range(newregistration_monthly['Year'].min(), newregistration_monthly['Year'].max() + 1)
major_ticks = [pd.to_datetime(f'{year}-01-01') for year in years]
ax.set_xticks(major_ticks)
ax.set_xticklabels([str(year) for year in years])

from matplotlib.dates import MonthLocator, DateFormatter
ax.xaxis.set_minor_locator(MonthLocator())

ax.grid(True, which='major', alpha=0.6, linewidth=1)  
ax.grid(True, which='minor', alpha=0.3, linewidth=0.5)  

ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))

events = [
    (datetime(2016, 5, 18), "Start Environmental Bonus"),
    (datetime(2019, 1, 1), "0.5% Rule (Company Cars)"),
    (datetime(2020, 1, 1), "0.25% Rule (Company Cars)"),
    (datetime(2023, 12, 17), "Abrupt End of Bonus"),
    (datetime(2022, 12, 31), "End of EV + PHEV subsidy")
]
y_pos = [15000, 30000, 50000, 105000, 0] # 60.000 auf 105.000 erhöht

for i, (date, text) in enumerate(events):
    if date >= electric_data['Date'].min() and date <= electric_data['Date'].max():
        
        plt.axvline(x=date, color='purple', linestyle=':', linewidth=1.5)
        
        plt.text(date, y_pos[i], text, 
                 rotation=0, color='purple', fontsize=9,
                 bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=0.2))

plt.xticks(rotation=0)  
plt.show()

# Correlation: Charging Stations vs New Registrations

In [ ]:
df_charge = pd.read_csv("EV CHARGING - Sheet1.csv", low_memory=False)
df_charge['Date'] = pd.to_datetime(df_charge['Inbetriebnahmedatum'], format='%d.%m.%Y', errors='coerce')  
df_charge = df_charge.dropna(subset=['Date'])
df_charge['Anzahl Ladepunkte'] = pd.to_numeric(df_charge['Anzahl Ladepunkte'], errors='coerce').fillna(0)   
df_charge['YearMonth'] = df_charge['Date'].dt.to_period('M')
    
monthly_new_charging = df_charge.groupby('YearMonth')['Anzahl Ladepunkte'].sum().reset_index()
monthly_new_charging.rename(columns={'Anzahl Ladepunkte': 'NewChargingPoints'}, inplace=True)
monthly_new_charging['Date'] = monthly_new_charging['YearMonth'].dt.to_timestamp()
    
df_regs = pd.read_csv("ev_registrations_cleaned.csv")
df_regs['Date'] = pd.to_datetime(df_regs['Year'].astype(str) + '-' + df_regs['Month'].astype(str) + '-01')

monthly_total_regs = df_regs.groupby('Date')['Count'].sum().reset_index()
monthly_total_regs.rename(columns={'Count': 'TotalNewEVs'}, inplace=True)
    
merged_monthly = pd.merge(monthly_new_charging[['Date', 'NewChargingPoints']], 
                              monthly_total_regs, 
                              on='Date', 
                              how='inner')
merged_monthly['Year'] = merged_monthly['Date'].dt.year

start_analysis_date = pd.to_datetime('2015-01-01') 

merged_monthly = merged_monthly[merged_monthly['Date'] >= start_analysis_date].copy()

slope, intercept, r_value, p_value, std_err = linregress(merged_monthly['NewChargingPoints'], merged_monthly['TotalNewEVs'])
print("slope:", slope, "intercept:", intercept, "r-value:", r_value, "p-value:", p_value, "std_err:", std_err)
        
plt.figure(figsize=(13, 7))
        
scatter = plt.scatter(merged_monthly['NewChargingPoints'], merged_monthly['TotalNewEVs'],
                              c=merged_monthly['Year'],
                              cmap='viridis',
                              label='Monthly Data Points (Color by Year)',
                              alpha=0.7, # Make points slightly transparent
                              s=50)
        
cbar = plt.colorbar(scatter)
cbar.set_label('Year', rotation=270, labelpad=15)
unique_years = sorted(merged_monthly['Year'].unique())
cbar.set_ticks(unique_years)

x_line = np.array([0, merged_monthly['NewChargingPoints'].max()]) 
y_line = intercept + slope * x_line
plt.plot(x_line, y_line, 'r-', 
        label=f'Regression Line (R²={r_value**2:.2f})')
        
plt.title('Monthly Correlation: New Charging Points vs New EV Registrations (Germany)', fontsize=14)
plt.xlabel('Number of New Public Charging Points Commissioned per Month', fontsize=12)
plt.ylabel('Total New EV (BEV + PHEV) Registrations per Month', fontsize=12)
        
formatter = ticker.FuncFormatter(lambda x, p: format(int(x), ','))
plt.gca().xaxis.set_major_formatter(formatter)
plt.gca().yaxis.set_major_formatter(formatter)
        
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
        
plt.show()

In [ ]:
X = merged_monthly[['NewChargingPoints']]
X = sm.add_constant(X)
y = merged_monthly['TotalNewEVs']
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
yearly_registrations = newregistration_monthly.groupby(['Year', 'Type'])['Count'].sum().reset_index()
yearly_totals = yearly_registrations.groupby('Year')['Count'].sum().reset_index()
yearly_totals.rename(columns={'Count': 'total_ev_registrations'}, inplace=True)

ratio_data = pd.merge(
    charging_ger.rename(columns={'Year': 'Year', 'Amount': 'new_charging_stations'}),
    yearly_totals,
    on='Year',
    how='inner'
)

ratio_data['stations_per_1000_registrations'] = (ratio_data['new_charging_stations'] / ratio_data['total_ev_registrations']) * 1000


fig, ax1 = plt.subplots(figsize=(12, 8))

bars = ax1.bar(ratio_data['Year'], ratio_data['stations_per_1000_registrations'], 
               color='green', alpha=0.7, label='Charging Stations per 1,000 EV + PHEV Registrations')
ax1.set_xlabel('Year')
ax1.set_ylabel('Charging Stations per 1,000 New EV + PHEV Registrations', color='black')
ax1.tick_params(axis='y', labelcolor='black')

for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{height:.1f}', ha='center', va='bottom', color='green', fontweight='bold')

ax2 = ax1.twinx()
ax2.plot(ratio_data['Year'], ratio_data['new_charging_stations'], 
         color='blue', marker='o', linewidth=2, markersize=6, label='New Charging Stations')
ax2.plot(ratio_data['Year'], ratio_data['total_ev_registrations']/1000, 
         color='red', marker='s', linewidth=2, markersize=6, label='New EV + PHEV Registrations (in thousands)')
ax2.set_ylabel('Absolute Numbers', color='black')

ax1.set_title('Charging Infrastructure vs. EV + PHEV Registrations Development in Germany', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()

for _, row in ratio_data.iterrows():
    print(f"{int(row['Year'])}: {row['stations_per_1000_registrations']:.1f} stations per 1,000 EV + PHEV registrations")

print(f"\nAverage ratio: {ratio_data['stations_per_1000_registrations'].mean():.1f} stations per 1,000 EV + PHEV registrations")
print(f"Trend: {'Improving' if ratio_data['stations_per_1000_registrations'].iloc[-1] > ratio_data['stations_per_1000_registrations'].iloc[0] else 'Declining'} infrastructure ratio")

correlation = ratio_data['new_charging_stations'].corr(ratio_data['total_ev_registrations'])
print(f"Correlation between new charging stations and EV + PHEV registrations: {correlation:.3f}")

# Infrastucture by States

In [ ]:
df = pd.read_csv("EV CHARGING - Sheet1.csv", low_memory=False)
df.head()

In [ ]:
bundesland_summary = df.groupby('Bundesland').size().reset_index(name='Anzahl')
bundesland_summary_sorted = bundesland_summary.sort_values(['Anzahl'], ascending=False)
print('Charging Stations by State:')
print(bundesland_summary_sorted)

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(bundesland_summary_sorted['Bundesland'], bundesland_summary_sorted['Anzahl'])
plt.xticks(rotation=45, ha='right')
plt.xlabel('State')
plt.ylabel('Number of Charging Stations')
plt.title('Charging Stations by State')
plt.tight_layout()
plt.show()

In [ ]:
df = pd.read_csv("EV CHARGING - Sheet1.csv", low_memory=False)

df['Jahr'] = pd.to_datetime(df['Inbetriebnahmedatum'], format='%d.%m.%Y', errors='coerce').dt.year

df_cleaned = df.dropna(subset=['Jahr'])
df_cleaned = df_cleaned[(df_cleaned['Jahr'] >= 2008) & (df_cleaned['Jahr'] <= datetime.now().year)] 

year_summary = df_cleaned.groupby('Jahr').size().reset_index(name='Anzahl')
plt.figure(figsize=(10, 6))
plt.bar(year_summary['Jahr'], year_summary['Anzahl'])
plt.xlabel('Year')
plt.ylabel('Number of Charging Stations')
plt.title('Charging Stations by Year of Commissioning')

plt.xlim(left=2008, right=year_summary['Jahr'].max() + 0.5)
plt.ylim(bottom=1)

plt.tight_layout()
plt.show()

In [ ]:
df['Jahr'] = pd.to_datetime(df['Inbetriebnahmedatum'], format='%d.%m.%Y', errors='coerce').dt.year
year_summary = df.groupby('Jahr').size().reset_index(name='Anzahl')
print('Charging stations by Year of Commissioning:')
print(year_summary)

In [ ]:
jahre_int = df['Jahr'].dropna().astype(int)
df_jahr_int = df[(jahre_int == df['Jahr']) & (df['Jahr'] >= 2018)]
years = sorted(df_jahr_int['Jahr'].unique())
pivot_line = df_jahr_int[df_jahr_int['Jahr'].isin(years)].pivot_table(index='Jahr', columns='Bundesland', values='Inbetriebnahmedatum', aggfunc='count', fill_value=0)
num_states = len(pivot_line.columns)
colors = cm.get_cmap('tab20', num_states)
plt.figure(figsize=(14,7))
for i, state in enumerate(pivot_line.columns):
    plt.plot(pivot_line.index, pivot_line[state], marker='o', label=state, color=colors(i))
plt.xlabel('Year')
plt.ylabel('Number of Charging Stations')
plt.title('Development of Charging Stations by State')
plt.xticks(years)
plt.legend(title='State', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
jahre_int = df['Jahr'].dropna().astype(int)
df_jahr_int = df[(jahre_int == df['Jahr']) & (df['Jahr'] >= 2018)]
years = sorted(df_jahr_int['Jahr'].unique())

pivot_cumulative = df_jahr_int[df_jahr_int['Jahr'].isin(years)].pivot_table(
    index='Jahr', columns='Bundesland', values='Inbetriebnahmedatum', 
    aggfunc='count', fill_value=0
)

pivot_cumulative = pivot_cumulative.cumsum()

num_states = len(pivot_cumulative.columns)
colors = cm.get_cmap('tab20', num_states)

plt.figure(figsize=(14,7))
for i, state in enumerate(pivot_cumulative.columns):
    plt.plot(pivot_cumulative.index, pivot_cumulative[state], 
             marker='o', label=state, color=colors(i), linewidth=2)

plt.xlabel('Year')
plt.ylabel('Cumulative Number of Charging Stations')
plt.title('Cumulative Development of Charging Stations by State')
plt.xticks(years)
plt.legend(title='State', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Forecasting

In [ ]:
df_csv = pd.read_csv("EV CHARGING - Sheet1.csv", low_memory=False)

df_csv['Date'] = pd.to_datetime(df_csv['Inbetriebnahmedatum'], format='%d.%m.%Y', errors='coerce')
df_csv = df_csv.dropna(subset=['Date'])

df_csv['Anzahl Ladepunkte'] = pd.to_numeric(df_csv['Anzahl Ladepunkte'], errors='coerce').fillna(0)
df_csv['Year'] = df_csv['Date'].dt.year

df_csv = df_csv[(df_csv['Year'] >= 2010) & (df_csv['Year'] <= 2025)]

points_per_year = df_csv.groupby('Year')['Anzahl Ladepunkte'].sum().reset_index()
points_per_year['Gesamtbestand'] = points_per_year['Anzahl Ladepunkte'].cumsum()

df = points_per_year[['Year', 'Gesamtbestand']].copy()
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-12-31')

last_real_date = df_csv['Date'].max()
df.loc[df.index[-1], 'Date'] = last_real_date
    
start_date = df['Date'].min()
df['Days'] = (df['Date'] - start_date).dt.days
    
X_all = df[['Days']]
y_all = df['Gesamtbestand']


current_date = df['Date'].max()
current_points = df['Gesamtbestand'].max()
target_date_goal = datetime(2030, 12, 31)
target_points_goal = 1000000
days_to_goal = (target_date_goal - current_date).days


future_days_rel = np.arange(0, days_to_goal + 1)
future_days_abs = future_days_rel + df['Days'].max() 
X_future = future_days_abs.reshape(-1, 1)
future_dates = [current_date + pd.Timedelta(days=int(d)) for d in future_days_rel]


#"Realistic" (Polynomic Regression)
poly_model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
poly_model.fit(X_all, y_all)
y_pred_poly = poly_model.predict(X_future)
y_pred_poly_anchored = y_pred_poly - y_pred_poly[0] + current_points
prediction_poly = y_pred_poly_anchored[-1]


#"Pessimistic" (Recency Heursistics)
p_last = df.iloc[-1]
p_second_last = df.iloc[-2]

days_diff = p_last['Days'] - p_second_last['Days']
points_diff = p_last['Gesamtbestand'] - p_second_last['Gesamtbestand']
daily_growth_recency = points_diff / days_diff

y_pred_recency = current_points + (daily_growth_recency * future_days_rel)
prediction_recency = y_pred_recency[-1]


#"Optimistic" (Exponential - Goal)
P_0 = current_points; P_t = target_points_goal; t = days_to_goal
r_required = np.log(P_t / P_0) / t
y_exp_required = P_0 * np.exp(r_required * future_days_rel) 
t_exp_dates_required = [current_date + pd.Timedelta(days=int(d)) for d in future_days_rel]



plt.figure(figsize=(15, 9))

plt.plot(df['Date'], df['Gesamtbestand'], 'b-', marker='o', 
         label='Actual Data (2014-2025)', linewidth=2.5, markersize=8)

plt.plot(future_dates, y_pred_poly_anchored, 'r--', 
         label=f'Scenario 1: "Realistic" (Polynomial) -> {int(prediction_poly):,}', 
        )

plt.plot(future_dates, y_pred_recency, 'c--', 
         label=f'Scenario 2: "Pessimistic" (Recency Heuristics) -> {int(prediction_recency):,}',
        )

plt.plot(t_exp_dates_required, y_exp_required, 'k--', 
         label=f'Scenario 3: "Optimistic" (Goal Path) -> {target_points_goal:,}',
        )

plt.axhline(y=target_points_goal, color='g', linestyle=':', label='Goal (1 Million)')

plt.title('Forecasting Scenarios and Political Events (Charging Stations)', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of Charging Points', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)

plt.axvline(x=current_date, color='grey', linestyle='--', label='Forecast Start (Aug 2025)')

events = [
    (datetime(2016, 5, 18), "Start Environmental Bonus"),
    (datetime(2020, 1, 1), "Start 0.25% Rule (Company Cars)"),
    (datetime(2022, 10, 19), "Masterplan II"),
    (datetime(2023, 12, 17), "End of Environmental Bonus"),
    (datetime(2022, 12, 31), "End of EV + PHEV subsidy")
]
y_pos = [200000, 300000, 400000, 600000, 500000] 

for i, (date, text) in enumerate(events):
    plt.axvline(x=date, color='purple', linestyle=':', linewidth=1)
    plt.text(date + pd.Timedelta(days=30), y_pos[i], text, 
             rotation=0, color='purple', fontsize=9,
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=0.2))

plt.legend(loc='upper left', fontsize=10, frameon=True, facecolor='white', framealpha=0.8)

formatter = ticker.FuncFormatter(lambda x, p: format(int(x), ','))
plt.gca().get_yaxis().set_major_formatter(formatter)

plt.xlim(left=datetime(2014, 1, 1), right=target_date_goal)
plt.ylim(bottom=-25000, top=target_points_goal * 1.1) 

plt.show()

In [ ]:
df_regs = pd.read_csv("ev_registrations_cleaned.csv")

df_ev = df_regs[df_regs['Type'] == 'Reine Elektroautos'].copy()

df_ev['Date'] = pd.to_datetime(df_ev['Year'].astype(str) + '-' + df_ev['Month'].astype(str) + '-01') + pd.offsets.MonthEnd(0)
df_ev = df_ev.sort_values('Date')

initial_stock_date = pd.to_datetime('2013-12-31')
initial_stock_value = 12156
df_ev['Cumulative_New_Regs'] = df_ev['Count'].cumsum()
df_ev['EV_Stock'] = initial_stock_value + df_ev['Cumulative_New_Regs']

df = df_ev[['Date', 'EV_Stock']].copy()
start_row = pd.DataFrame({'Date': [initial_stock_date], 'EV_Stock': [initial_stock_value]})
df = pd.concat([start_row, df], ignore_index=True)

start_date_model = df['Date'].min()
df['Days'] = (df['Date'] - start_date_model).dt.days

X_all = df[['Days']]
y_all = df['EV_Stock']

current_date = df['Date'].max()
current_points = df['EV_Stock'].max()
target_date_goal = datetime(2030, 12, 31)
target_points_goal = 15_000_000 
days_to_goal = (target_date_goal - current_date).days

future_days_rel = np.arange(0, days_to_goal + 1)
future_days_abs = future_days_rel + df['Days'].max()
X_future = future_days_abs.reshape(-1, 1)
future_dates = [current_date + pd.Timedelta(days=int(d)) for d in future_days_rel]

# "Realistic" (Polynomial)
poly_model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
poly_model.fit(X_all, y_all)
y_pred_poly = poly_model.predict(X_future)
y_pred_poly_anchored = y_pred_poly - y_pred_poly[0] + current_points
prediction_poly = y_pred_poly_anchored[-1]

# "Pessimistic" (Recency Heuristic)
df_last_12m = df.tail(13)
monthly_increase = df_last_12m['EV_Stock'].diff().mean()
daily_growth_recency_strict = monthly_increase / 30.44 
y_pred_recency = current_points + (daily_growth_recency_strict * future_days_rel)
prediction_recency = y_pred_recency[-1]

# "Optimistic" (Exponential Goal Path)
P_0 = current_points
P_t = target_points_goal
t = days_to_goal
if P_0 <= 0: P_0 = 1 
r_required = np.log(P_t / P_0) / t
y_exp_required = P_0 * np.exp(r_required * future_days_rel)
t_exp_dates_required = [current_date + pd.Timedelta(days=int(d)) for d in future_days_rel]


plt.figure(figsize=(15, 9))

plt.plot(df['Date'], df['EV_Stock'], 'b-', marker='.', markersize=5,
         label='EVs (Cumulated)', linewidth=2.5)

plt.plot(future_dates, y_pred_poly_anchored, 'r--',
         label=f'Scenario 1: "Realistic" (Polynomial Regresssion) -> {int(prediction_poly):,}')

plt.plot(future_dates, y_pred_recency, 'c--',
         label=f'Scenario 2: "Pessimistic" (Recency Heuristics) -> {int(prediction_recency):,}')

plt.plot(t_exp_dates_required, y_exp_required, 'k--',
         label=f'Scenario 3: "Optimistic" (Goal Path) -> {target_points_goal:,}')

plt.axhline(y=target_points_goal, color='g', linestyle=':', label='Goal (15M EVs)') 

plt.title('EVs Forecast Scenarios and Political Events (EVs)', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('EVs', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.axvline(x=current_date, color='grey', linestyle='--', label='Forecast Start')

events = [
    (datetime(2016, 5, 18), "Start Environmental Bonus"),
    (datetime(2020, 1, 1), "Start 0.25% Rule (Company Cars)"),
    (datetime(2022, 10, 19), "Masterplan II"),
    (datetime(2023, 12, 17), "End of Environmental Bonus"),
    (datetime(2022, 12, 31), "End of EV + PHEV subsidy")
]
y_pos = [2000000, 4000000, 6000000, 10000000, 8000000] 

for i, (date, text) in enumerate(events):
    plt.axvline(x=date, color='purple', linestyle=':', linewidth=1)
    plt.text(date + pd.Timedelta(days=30), y_pos[i], text, 
             rotation=0, color='purple', fontsize=9,
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=0.2))

plt.legend(loc='upper left', fontsize=10, frameon=True, facecolor='white', framealpha=0.8)
formatter = ticker.FuncFormatter(lambda x, p: f'{x/1_000_000:.1f}M') 
plt.gca().get_yaxis().set_major_formatter(formatter)
plt.xlim(left=datetime(2014, 1, 1), right=target_date_goal) 
plt.ylim(bottom=0, top=target_points_goal * 1.1)

plt.show()